In [ ]:
# !pip install mljar-supervised

In [10]:
import os
import time
from os import path

import joblib
import numpy as np
import pandas as pd
from autogluon.tabular import TabularPredictor
from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.feature_selection import SelectKBest, VarianceThreshold, f_classif
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.metrics import balanced_accuracy_score, mutual_info_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
from supervised.automl import AutoML  # mljar-supervised

c:\JAN_CICHOMSKI\STUDIA\STUDIA_SEMESTR_7_2023_ZIMA\auto_ml\homeworks\homework_1\AUTOML_HM2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Constants

SEED = 42
N_JOBS = -1
RANDOM_SEARCH_N_ITER = 50
TRAIN_TIME_LIMIT_AUTOGLUON = 60 * 30
TRAIN_TIME_LIMIT_MLJAR = 60 * 30
TRAIN_TIME_LIMIT_AUTO_SKLEARN = 60 * 30
OUTPUT_DIR_MANUAL = path.join("output", "manual")
OUTPUT_DIR_AUTOGLUON = path.join("output", "autogluon")
OUTPUT_DIR_MLJAR = path.join("output", "mljar")
OUTPUT_DIR_AUTO_SKLEARN = path.join("output", "auto_sklearn")
UNIQUE_ID = time.strftime("%Y%m%d_%H%M%S")
APPLY_REMOVE_LOW_VARIANCE_FEATURES = True
APPLY_REMOVE_CORRELATED_FEATURES = True
APPLY_REMOVE_RANDOM_FEATURES = False
APPLY_ANOVA = True
ANOVE_FEATURES = 15

In [12]:
# prepare output directories
for output_dir in [
    OUTPUT_DIR_MANUAL,
    OUTPUT_DIR_AUTOGLUON,
    OUTPUT_DIR_MLJAR,
    OUTPUT_DIR_AUTO_SKLEARN,
]:
    if not path.exists(path.join(output_dir, UNIQUE_ID)):
        print(f"Creating output directory {path.join(output_dir, UNIQUE_ID)}")
        os.makedirs(path.join(output_dir, UNIQUE_ID))

Creating output directory output\manual\20240112_090129
Creating output directory output\autogluon\20240112_090129
Creating output directory output\mljar\20240112_090129
Creating output directory output\auto_sklearn\20240112_090129


In [13]:
# Remove Highly Correlated Columns
# def remove_highly_correlated_features(train_x, valid_x, test_x, threshold=0.95):
#     corr_matrix = np.corrcoef(train_x, rowvar=False)
#     upper = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
#     to_drop = np.where(np.abs(corr_matrix[upper]) > threshold)[0]
#     print(to_drop)
#     train_x = np.delete(train_x, to_drop, axis=1)
#     valid_x = np.delete(valid_x, to_drop, axis=1)
#     test_x = np.delete(test_x, to_drop, axis=1)
#     return train_x, valid_x, test_x


def remove_highly_correlated_features(train_x, valid_x, test_x, threshold=0.95):
    # Calculate correlation matrix
    corr_matrix = np.corrcoef(train_x, rowvar=False)
    # Select upper triangle of correlation matrix
    upper = np.triu(corr_matrix, k=1)
    # Find indices of feature columns with correlation greater than threshold
    to_drop = [i for i in range(upper.shape[1]) if any(upper[:, i] > threshold)]

    # Drop features from train, validation, and test set
    train_x = np.delete(train_x, to_drop, axis=1)
    valid_x = np.delete(valid_x, to_drop, axis=1)
    test_x = np.delete(test_x, to_drop, axis=1)

    return train_x, valid_x, test_x


# pandas
# # Remove Highly Correlated Columns
# def remove_highly_correlated_features(train_x, valid_x, text_x, threshold=0.95):
#     corr_matrix = train_x.corr().abs()
#     # Select upper triangle of correlation matrix
#     upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
#     # Find index of feature columns with correlation greater than threshold
#     to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
#     train_x = train_x.drop(to_drop, axis=1)
#     valid_x = valid_x.drop(to_drop, axis=1)
#     text_x = text_x.drop(to_drop, axis=1)
#     return train_x, valid_x, text_x

In [14]:
# Remove Low Variance Columns
def remove_low_variance_features(train_x, valid_x, test_x, threshold=(0.8 * (1 - 0.8))):
    sel = VarianceThreshold(threshold=threshold)
    sel.fit(train_x)
    train_x = train_x[:, sel.get_support(indices=True)]
    valid_x = valid_x[:, sel.get_support(indices=True)]
    test_x = test_x[:, sel.get_support(indices=True)]
    return train_x, valid_x, test_x

In [15]:
# Remove Random Columns
def remove_random_features(
    train_x: np.ndarray,
    train_y: np.ndarray,
    valid_x: np.ndarray,
    test_x: np.ndarray,
    importance=0.005,
):
    tree: DecisionTreeClassifier = DecisionTreeClassifier(random_state=0)
    tree.fit(train_x, train_y)
    importances = tree.feature_importances_

    # Assume columns with very low importance are "random"
    # This threshold can be adjusted based on domain knowledge
    important_indices = [i for i, imp in enumerate(importances) if imp > importance]
    train_x = train_x[:, important_indices]
    valid_x = valid_x[:, important_indices]
    test_x = test_x[:, important_indices]
    return train_x, valid_x, test_x

In [16]:
def anova_filter(
    train_x: np.ndarray,
    train_y: np.ndarray,
    valid_x: np.ndarray,
    test_x: np.ndarray,
    k: int = 50,
):
    # Using ANOVA F-test to select features
    selector = SelectKBest(
        f_classif, k=k
    )  # Change k to select the number of features you want
    selector.fit(train_x, train_y)

    # Get F-values and p-values for each feature
    # f_values = selector.scores_
    # p_values = selector.pvalues_

    # Selecting features (you can use a threshold or select top k features)
    # selected_features = train_x.columns[selector.get_support()]

    # Transforming train_x to include only the selected features
    train_x = selector.transform(train_x)
    valid_x = selector.transform(valid_x)
    test_x = selector.transform(test_x)
    return train_x, valid_x, test_x

In [17]:
prefix = ""

_test_x = pd.read_table(prefix + "artificial_test.data", sep=" ", header=None)
_test_x.drop(_test_x.columns[500], axis=1, inplace=True)
_train_y = pd.read_table(prefix + "artificial_train.labels", header=None)
_train_x = pd.read_table(prefix + "artificial_train.data", sep=" ", header=None)
_train_x.drop(_train_x.columns[500], axis=1, inplace=True)

In [18]:
_train_x, _train_y = shuffle(_train_x, _train_y, random_state=42)

In [19]:
def get_train_and_validation_data():
    split = 400
    train_x, valid_x = _train_x[split:].values, _train_x[:split].values
    train_y, valid_y = _train_y[split:].values, _train_y[:split].values
    return train_x, train_y, valid_x, valid_y

In [20]:
train_x, train_y, valid_x, valid_y = get_train_and_validation_data()
train_y = train_y.reshape(-1, 1)
valid_y = valid_y.reshape(-1, 1)
print(train_x.shape, train_y.shape, valid_x.shape, valid_y.shape)

(1600, 500) (1600, 1) (400, 500) (400, 1)


In [21]:
if APPLY_REMOVE_CORRELATED_FEATURES:
    train_x, valid_x, test_x = remove_highly_correlated_features(
        train_x, valid_x, _test_x
    )
    print("train_x.shape: ", train_x.shape)

train_x.shape:  (1600, 490)


In [22]:
if APPLY_REMOVE_LOW_VARIANCE_FEATURES:
    train_x, valid_x, test_x = remove_low_variance_features(train_x, valid_x, test_x)
    print("train_x.shape: ", train_x.shape)

train_x.shape:  (1600, 490)


In [23]:
if APPLY_REMOVE_RANDOM_FEATURES:
    train_x, valid_x, test_x = remove_random_features(
        train_x=train_x, train_y=train_y, valid_x=valid_x, test_x=test_x
    )
    print("train_x.shape: ", train_x.shape)

In [24]:
if APPLY_ANOVA:
    train_x, valid_x, test_x = anova_filter(
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        test_x=test_x,
        k=ANOVE_FEATURES,
    )
    print("train_x.shape: ", train_x.shape)

train_x.shape:  (1600, 15)


c:\JAN_CICHOMSKI\STUDIA\STUDIA_SEMESTR_7_2023_ZIMA\auto_ml\homeworks\homework_1\AUTOML_HM2\.venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [25]:
# convert to dataframe
# train_x = pd.DataFrame(train_x)
# valid_x = pd.DataFrame(valid_x)

In [26]:
print("train_x.shape: ", train_x.shape)
print("train_y.shape: ", train_y.shape)

train_x.shape:  (1600, 15)
train_y.shape:  (1600, 1)


In [27]:
# label = "class"
# train_y = train_y.rename(columns={0: label})
# valid_y = valid_y.rename(columns={0: label})
# train_data = pd.concat([train_x, train_y[label]], axis=1)

In [28]:
# sanity check
(
    original_train_x,
    original_train_y,
    original_valid_x,
    original_valid_y,
) = get_train_and_validation_data()
for y, original_y in zip([train_y, valid_y], [original_train_y, original_valid_y]):
    assert y.shape == original_y.shape

### manual model

In [29]:
# train_y = train_y.ravel()
# valid_y = valid_y.ravel()

In [31]:
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import RandomizedSearchCV

base_classifiers_1 = [
    (
        "rf",
        make_pipeline(
            StandardScaler(),
            RandomForestClassifier(n_estimators=100, random_state=SEED),
        ),
    ),
    # ("svc", make_pipeline(StandardScaler(), SVC(random_state=SEED))),
    ("dt", make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=SEED))),
    # (
    #     "elasticnet",
    #     make_pipeline(
    #         StandardScaler(),
    #         ElasticNet(
    #             alpha=0.0001, l1_ratio=0.15, max_iter=1000, tol=1e-3, random_state=SEED
    #         ),
    #     ),
    # ),
    (
        "mlp",
        make_pipeline(
            StandardScaler(),
            MLPClassifier(
                random_state=SEED,
                max_iter=1000,
                tol=1e-3,
                hidden_layer_sizes=(100, 300, 200, 100),
            ),
        ),
    ),
    (
        "gbc",
        make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=SEED)),
    ),
]

# First Stacking Layer
first_layer = StackingClassifier(
    estimators=base_classifiers_1, final_estimator=LogisticRegression(), cv=5
)

base_classifiers_2 = [
    ("first_layer", first_layer),
    (
        "mlp",
        make_pipeline(
            StandardScaler(),
            MLPClassifier(
                random_state=SEED,
                max_iter=1000,
                tol=1e-3,
                hidden_layer_sizes=(100, 300, 200, 100),
            ),
        ),
    ),
    (
        "gbc",
        make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=SEED)),
    ),
]


param_distributions = {
    # Parameters for the First Stacking Layer
    "stacked_ensemble__first_layer__rf__randomforestclassifier__n_estimators": randint(
        50, 200
    ),
    "stacked_ensemble__first_layer__rf__randomforestclassifier__max_depth": randint(
        15, 50
    ),
    "stacked_ensemble__first_layer__rf__randomforestclassifier__min_samples_split": randint(
        4, 20
    ),
    "stacked_ensemble__first_layer__rf__randomforestclassifier__min_samples_leaf": randint(
        4, 20
    ),
    # "first_layer__svc__svc__C": uniform(0.1, 10),
    # "first_layer__svc__svc__gamma": ["scale", "auto"],
    "stacked_ensemble__first_layer__dt__decisiontreeclassifier__max_depth": randint(
        15, 50
    ),
    "stacked_ensemble__first_layer__dt__decisiontreeclassifier__min_samples_split": randint(
        5, 20
    ),
    "stacked_ensemble__first_layer__dt__decisiontreeclassifier__min_samples_leaf": randint(
        5, 20
    ),
    # "first_layer__elasticnet__elasticnet__alpha": uniform(0.0001, 1),
    # "first_layer__elasticnet__elasticnet__l1_ratio": uniform(0, 1),
    "stacked_ensemble__first_layer__mlp__mlpclassifier__alpha": uniform(0.0001, 1),
    "stacked_ensemble__first_layer__mlp__mlpclassifier__learning_rate_init": uniform(
        0.001, 0.1
    ),
    "stacked_ensemble__first_layer__mlp__mlpclassifier__hidden_layer_sizes": [
        (50, 40),
        (50, 100, 50),
        (50, 150, 100, 50),
    ],
    # Parameters for the Second Stacking Layer
    "stacked_ensemble__mlp__mlpclassifier__alpha": uniform(0.0001, 1),
    "stacked_ensemble__mlp__mlpclassifier__learning_rate_init": uniform(0.001, 0.1),
    "stacked_ensemble__mlp__mlpclassifier__hidden_layer_sizes": [
        (50, 40),
        (50, 100, 50),
        (50, 150, 100, 50),
    ],
    "stacked_ensemble__gbc__gradientboostingclassifier__n_estimators": randint(50, 200),
    "stacked_ensemble__gbc__gradientboostingclassifier__max_depth": randint(15, 50),
    "stacked_ensemble__gbc__gradientboostingclassifier__min_samples_split": randint(
        4, 20
    ),
    "stacked_ensemble__gbc__gradientboostingclassifier__min_samples_leaf": randint(
        4, 20
    ),
    # Parameters for the Final Estimator
    "stacked_ensemble__final_estimator__C": uniform(0.01, 10),
    "gbc__n_estimators": randint(50, 200),
    "gbc__max_depth": randint(15, 50),
    "gbc__min_samples_split": randint(4, 20),
    "gbc__min_samples_leaf": randint(4, 20),
    "rf__n_estimators": randint(50, 200),
    "rf__max_depth": randint(15, 50),
    "rf__min_samples_split": randint(4, 20),
    "rf__min_samples_leaf": randint(4, 20),
}


# Second Stacking Layer
stacked_ensemble_model = StackingClassifier(
    estimators=base_classifiers_2, final_estimator=LogisticRegression(), cv=5
)
# Define the committee of models
committee_models = [
    ("stacked_ensemble", stacked_ensemble_model),
    (
        "gbc",
        GradientBoostingClassifier(random_state=SEED, n_estimators=100, max_depth=15),
    ),
    ("rf", RandomForestClassifier(random_state=SEED, n_estimators=100, max_depth=15)),
    (
        "mlp",
        MLPClassifier(
            random_state=SEED,
            max_iter=1000,
            tol=1e-3,
            hidden_layer_sizes=(100, 300, 200, 100),
        ),
    ),
]

# Create the committee model
committee_model = VotingClassifier(committee_models, voting="soft")
# print(committee_model)
# Perform randomized search
random_search = RandomizedSearchCV(
    committee_model,
    param_distributions=param_distributions,
    scoring="balanced_accuracy",
    n_iter=RANDOM_SEARCH_N_ITER,
    cv=5,
    verbose=4,
    random_state=SEED,
    n_jobs=8,
)

random_search.fit(train_x, train_y.ravel())

y_pred = random_search.predict(valid_x)

balanced_accuracy = balanced_accuracy_score(valid_y, y_pred)

print(f"Model Balanced Accuracy: {balanced_accuracy}")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Model Balanced Accuracy: 0.795328949013528


In [ ]:
print(random_search.best_params_)

In [33]:
proba = random_search.predict_proba(test_x)
output_path = path.join(OUTPUT_DIR_MANUAL, UNIQUE_ID, "manual_model.txt")
np.savetxt(output_path, proba[:, 1], delimiter="\n")
joblib.dump(
    random_search, path.join(OUTPUT_DIR_MANUAL, UNIQUE_ID, "manual_model_pred.pkl")
)
# random_search.save(path.join(OUTPUT_DIR_MANUAL, UNIQUE_ID, "manual_model.pkl"))

AttributeError: 'RandomizedSearchCV' object has no attribute 'save'

### Autogluon

In [ ]:
print(train_x.shape, train_y.shape)
print(valid_x.shape, valid_y.shape)

In [ ]:
train_data = np.concatenate((train_x, train_y), axis=1)
train_data_pd = pd.DataFrame(train_data)
train_data_pd.rename(columns={train_data_pd.columns[-1]: "class"}, inplace=True)

valid_data = np.concatenate((valid_x, valid_y), axis=1)
valid_data_pd = pd.DataFrame(valid_data)
valid_data_pd.rename(columns={valid_data_pd.columns[-1]: "class"}, inplace=True)

In [ ]:
print(train_data_pd.shape, valid_data_pd.shape)

In [ ]:
save_path = path.join(OUTPUT_DIR_AUTOGLUON, UNIQUE_ID)
predictor = TabularPredictor(
    label="class",
    path=save_path,
    eval_metric="balanced_accuracy",
    problem_type="binary",
).fit(
    train_data_pd,
    time_limit=TRAIN_TIME_LIMIT_AUTOGLUON,
    presets="best_quality",
    hyperparameters="default",
)

In [ ]:
predictor.leaderboard()

In [ ]:
predictor.evaluate(valid_data_pd)

In [ ]:
proba = predictor.predict_proba(pd.DataFrame(test_x))
output_path = path.join(OUTPUT_DIR_AUTOGLUON, UNIQUE_ID, "manual_model_pred.txt")
np.savetxt(output_path, proba.values[:, 1], delimiter="\n")
# predictor.save(path.join(OUTPUT_DIR_AUTOGLUON, UNIQUE_ID, "manual_model.pkl"))

### MLJar

In [ ]:
automl = AutoML(
    mode="Compete",
    ml_task="binary_classification",
    total_time_limit=TRAIN_TIME_LIMIT_MLJAR,
    eval_metric="f1",
    random_state=SEED,
    results_path=path.join(OUTPUT_DIR_MLJAR, UNIQUE_ID),
)

automl.fit(train_x, train_y.ravel())

In [ ]:
print(valid_x.shape, valid_y.shape)
print(train_x.shape, train_y.shape)
predictions = automl.predict(valid_x)
score = balanced_accuracy_score(valid_y, predictions)
print(f"Model Balanced Accuracy: {score}")

In [ ]:
proba = automl.predict_proba(test_x)
output_path = path.join(OUTPUT_DIR_MLJAR, UNIQUE_ID, "mljar_model_proba.txt")
np.savetxt(output_path, proba[:, 1], delimiter="\n")

In [ ]:
print(proba)

In [ ]:
def ensemble_predict(X, model1, model2):
    pred1 = model1.predict_proba(pd.DataFrame(X)).values[:, 1]
    pred2 = model2.predict_proba(X)[:, 1]
    # pred3 = model3.predict_proba(X)[:,1]
    print(pred1, pred2)
    avg_pred = (pred1 + pred2) / 2

    return avg_pred


# Example of using the ensemble
final_predictions = ensemble_predict(test_x, predictor, automl)

In [ ]:
print(final_predictions)
os.makedirs(path.join("ensamble", UNIQUE_ID), exist_ok=True)


np.savetxt(
    path.join("ensamble", UNIQUE_ID, "123manual_model_pred.txt"),
    final_predictions,
    delimiter="\n",
)

### Auto SKLearn

In [ ]:
# !pip install auto-sklearn
# !pip install ydata-profiling
# from autosklearn.classification import AutoSklearnClassifier
from autosklearn.experimental.askl2 import AutoSklearn2Classifier
from autosklearn.metrics import balanced_accuracy

In [ ]:
settings = {
    "time_left_for_this_task": TRAIN_TIME_LIMIT_AUTO_SKLEARN,
    "seed": SEED,
    "metric": balanced_accuracy,
    "n_jobs": -1,
}

In [ ]:
askl2 = AutoSklearn2Classifier(**settings)
askl2.fit(train_x, train_y)

In [ ]:
leaderboard = askl2.leaderboard(sort_by="model_id", ensemble_only=True)
print(leaderboard)

In [ ]:
predictions = askl2.predict(valid_x)
balanced_accuracy_score(valid_y, predictions)

In [ ]:
proba = askl2.predict_proba(test_x)
output_path = path.join(OUTPUT_DIR_AUTO_SKLEARN, UNIQUE_ID, "manual_model.txt")
np.savetxt(output_path, proba, delimiter="\n")
askl2.save(path.join(OUTPUT_DIR_AUTO_SKLEARN, UNIQUE_ID, "manual_model.pkl"))